#### 1. Import pandas library

In [1]:
import pandas as pd


#### 2. Import users table:

In [2]:
 
#read the csv file and store it in 'users'
users = pd.read_csv(r'users_table.csv')
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 3. Rename Id column to userId

In [4]:

#rename the ID column to userId with rename() method
users = users.rename(columns={'Id':'userId'})

#test it worked
users.head()

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 4. Import posts table:

In [5]:
 #read the posts csv file and store it in 'posts'

posts = pd.read_csv(r'posts_table.csv')
posts.head()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [7]:

#rename the colums with rename() method

posts = posts.rename(columns={'Id':'postId', 'OwnerUserId':'userId'})

#test it worked
posts.head()

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [10]:

#create a dataframe with all columns for Users
usersdf = pd.DataFrame(users)

#create a dataframe with selected columns
usersdf2 = usersdf[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]


#test it with head method
usersdf2.head()





,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


In [11]:
#create a dataframe with all columns for Posts
postsdf = pd.DataFrame(posts)

#create a dataframe with selected columns
postsdf2 = postsdf[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

#test it with head method
postsdf2.head()

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [12]:
#merge the dataframes using the concatentate method as they don't have a matching column 
users_and_posts_df = pd.concat([usersdf2, postsdf2], axis = 1)

#test it using head(). It has worked - but not sure why integer values are now floats for post part of the table
users_and_posts_df.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,userId,ViewCount,CommentCount
0,-1,1,0,5007,1920,1.0,23.0,8.0,1278.0,1.0
1,2,101,25,3,0,2.0,22.0,24.0,8198.0,1.0
2,3,101,22,19,0,3.0,54.0,18.0,3613.0,4.0
3,4,101,11,0,0,4.0,13.0,23.0,5224.0,2.0
4,5,6792,1145,662,5,5.0,81.0,23.0,NaN,3.0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [26]:
#count number of nulls for each colum using the isnull() method and adding them all up with the sum() method
users_and_posts_df.isnull().sum() 

#they are all from the original post table columns

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId            325
Score             325
userId           1363
ViewCount       24430
CommentCount      325
dtype: int64

In [13]:
#testing this looks right
users_and_posts_df

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,userId,ViewCount,CommentCount
0,-1,1,0,5007,1920,1.0,23.0,8.0,1278.0,1.0
1,2,101,25,3,0,2.0,22.0,24.0,8198.0,1.0
2,3,101,22,19,0,3.0,54.0,18.0,3613.0,4.0
3,4,101,11,0,0,4.0,13.0,23.0,5224.0,2.0
4,5,6792,1145,662,5,5.0,81.0,23.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...
40320,55743,1,0,0,0,NaN,NaN,NaN,NaN,NaN
40321,55744,6,1,0,0,NaN,NaN,NaN,NaN,NaN
40322,55745,101,0,0,0,NaN,NaN,NaN,NaN,NaN
40323,55746,106,1,0,0,NaN,NaN,NaN,NaN,NaN


In [15]:
#testing this looks right

users_and_posts_df.tail()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,userId,ViewCount,CommentCount
40320,55743,1,0,0,0,NaN,NaN,NaN,NaN,NaN
40321,55744,6,1,0,0,NaN,NaN,NaN,NaN,NaN
40322,55745,101,0,0,0,NaN,NaN,NaN,NaN,NaN
40323,55746,106,1,0,0,NaN,NaN,NaN,NaN,NaN
40324,55747,1,0,0,0,NaN,NaN,NaN,NaN,NaN


#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [33]:
#fill all null values with 0 using the fillna() method so we don't delete users who haven't posted
fill_users_and_posts_df = users_and_posts_df.fillna(0)

#testing it. As we can see the bottom 5 rows were origionally null(showing NaN) now show 0.0
fill_users_and_posts_df.tail()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,userId,ViewCount,CommentCount
40320,55743,1,0,0,0,0.0,0.0,0.0,0.0,0.0
40321,55744,6,1,0,0,0.0,0.0,0.0,0.0,0.0
40322,55745,101,0,0,0,0.0,0.0,0.0,0.0,0.0
40323,55746,106,1,0,0,0.0,0.0,0.0,0.0,0.0
40324,55747,1,0,0,0,0.0,0.0,0.0,0.0,0.0


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [20]:
#data types are already numerical which is what I need to repalce with zero - not sure what datatype would avoid future issues of getting a null value?

#use dtypes method to see the current datatypes
fill_users_and_posts_df.dtypes


postId          float64
Score           float64
userId            int64
userId          float64
ViewCount       float64
CommentCount    float64
dtype: object